In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from bert_score import score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer
from IPython.display import display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
##555
from local_bert_scorer import LocalBERTScorer

In [2]:
MODEL_DIR = '/project/lt200301-edubot/Capstone-TamTanai/models'
INFERENCE_RESULT_DIR = '/project/lt200301-edubot/Capstone-TamTanai/inference_result'
LOG_DIR = '/project/lt200301-edubot/Capstone-TamTanai/logs'
dataset_DIR = "../asset/dataset/testdata.csv"

In [3]:
model_type = '/project/lt200301-edubot/bert_score/models/microsoft/deberta-xlarge-mnli'

bert_scorer = LocalBERTScorer(model_type=model_type, num_layers=40)

In [12]:
def find_best_model(model_name):
    log_path = os.path.join(LOG_DIR, f'{model_name}.csv')
    log = pd.read_csv(log_path)
    step = int(log.iloc[log['eval_loss'].argmin()]['step'])
    return os.path.join(MODEL_DIR, model_name, f'checkpoint-{step}')


def run_evaluation(MODEL_NAME, FIND_BEST_MODEL):
    MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, MODEL_NAME )
    if FIND_BEST_MODEL:
        MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
    INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}_fewshot.csv')
    df = pd.read_csv(INFERECE_RESULT_PATH)
    #df['response'] = df['response_finetune'].astype(str)
    df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    #df['response'] = df['response'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    df_dataset = pd.read_csv(dataset_DIR)
    
    answers = df['answer'].tolist()
    responses = df['response'].tolist()
    
    precision, recall, f1 = bert_scorer.score(responses, answers, verbose=True, batch_size=16)
    #print('\nBert Score')
    #print("average time:", df["time_finetune"].mean())
    print(f'Precision: {precision.mean().item()}')
    print(f'Recall:    {recall.mean().item()}')
    print(f'F1:        {f1.mean().item()}')

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
    tokenizer.model_max_length = 1000000000
    rouge = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
    for answer, response in tqdm(zip(answers, responses), total=len(answers)):
        scores = scorer.score(answer, response)
        for k in rouge:
            rouge[k].append(scores[k].fmeasure)
    print('\nRouge')
    for k in rouge:
        print(f'{k}: {np.mean(rouge[k])}')

    df['f1'] = f1
    df["Precison"] = precision
    df["Recall"] = recall
    df.to_csv("check_fewshot.csv")
    df = df.sort_values('f1')
    for _, row in df.iloc[:].iterrows():
        print('\nF1:', row['f1'])
        print('Q:', row['question'])
        print("K:", df_dataset[df_dataset["question"]==row["question"]]["knowledges"].values[0])
        print('A:', row['answer'])
        print('R:', row['response'])
        print()

In [11]:
def run_evaluation(MODEL_NAME, FIND_BEST_MODEL): ##วัด GPT-4o
    MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, 'Llama-3-Typhoon-1.5-8B-finetune-dataset-v1' )
    if FIND_BEST_MODEL:
        MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
    INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}.csv')
    df = pd.read_csv("../notebooks/output_new_test_2.csv")
    #df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    df['response_gpt'] = df['output_gpt'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    df_dataset = pd.read_csv(dataset_DIR)
    
    answers = df['answer'].tolist()
    responses = df['response_gpt'].tolist()
    precision, recall, f1 = bert_scorer.score(responses, answers, verbose=True, batch_size=16)
    #print('\nBert Score')
    #print("average time:", df["time_finetune"].mean())
    print(f'Precision: {precision.mean().item()}')
    print(f'Recall:    {recall.mean().item()}')
    print(f'F1:        {f1.mean().item()}')
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
    tokenizer.model_max_length = 1000000000
    rouge = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
    for answer, response in tqdm(zip(answers, responses), total=len(answers)):
        scores = scorer.score(answer, response)
        for k in rouge:
            rouge[k].append(scores[k].fmeasure)
    print('\nRouge')
    for k in rouge:
        print(f'{k}: {np.mean(rouge[k])}')

In [15]:
def run_evaluation(MODEL_NAME, FIND_BEST_MODEL): ## ดูผล Output
    MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, MODEL_NAME )
    if FIND_BEST_MODEL:
        MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
    INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}.csv')
    df = pd.read_csv(INFERECE_RESULT_PATH)
    #df['response'] = df['response_finetune'].astype(str)
    df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    #df['response'] = df['response'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    df_dataset = pd.read_csv(dataset_DIR)
    
    answers = df['answer'].tolist()
    responses = df['response'].tolist()
    
    precision, recall, f1 = bert_scorer.score(responses, answers, verbose=True, batch_size=16)
    #print('\nBert Score')
    df['f1'] = f1
    df = df.sort_values('f1')
    percentile_25 = np.percentile(df['f1'], 25)
    percentile_75 = np.percentile(df['f1'], 75)

    # แบ่ง DataFrame ตามช่วงเปอร์เซ็นไทล์
    df_0_25 = df[df['f1'] <= percentile_25]
    df_25_75 = df[(df['f1'] > percentile_25) & (df['f1'] <= percentile_75)]
    df_75_100 = df[df['f1'] > percentile_75]

    # สุ่มตัวอย่างจากแต่ละช่วง
    sample_0_25 = df_0_25.sample(n=25, random_state=2024)
    sample_25_75 = df_25_75.sample(n=10, random_state=2024)
    sample_75_100 = df_75_100.sample(n=5, random_state=2024)
    print("Percentile 0-25")
    for _, row in sample_0_25.iloc[:].iterrows():
        print('\nF1:', row['f1'])
        print('Q:', row['question'])
        print("K:", df_dataset[df_dataset["question"]==row["question"]]["knowledges"].values[0])
        print('A:', row['answer'])
        print('R:', row['response'])
        print()

    print("Percentile 25-75")
    for _, row in sample_25_75.iloc[:].iterrows():
        print('\nF1:', row['f1'])
        print('Q:', row['question'])
        print("K:", df_dataset[df_dataset["question"]==row["question"]]["knowledges"].values[0])
        print('A:', row['answer'])
        print('R:', row['response'])
        print()

    print("Percentile 75-100")
    for _, row in sample_75_100.iloc[:].iterrows():
        print('\nF1:', row['f1'])
        print('Q:', row['question'])
        print("K:", df_dataset[df_dataset["question"]==row["question"]]["knowledges"].values[0])
        print('A:', row['answer'])
        print('R:', row['response'])
        print()

In [12]:
def run_evaluation(MODEL_NAME, FIND_BEST_MODEL): ## ดูผล Output
    MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, MODEL_NAME )
    if FIND_BEST_MODEL:
        MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
    INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}.csv')
    df = pd.read_csv(INFERECE_RESULT_PATH)
    #df['response'] = df['response_finetune'].astype(str)
    df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    #df['response'] = df['response'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)
    df_dataset = pd.read_csv(dataset_DIR)
    
    answers = df['answer'].tolist()
    responses = df['response'].tolist()
    
    precision, recall, f1 = bert_scorer.score(responses, answers, verbose=True, batch_size=16)
    #print('\nBert Score')
    df['f1'] = f1
    df = df.sort_values('f1')
    percentile_25 = np.percentile(df['f1'], 25)
    percentile_75 = np.percentile(df['f1'], 75)

    # แบ่ง DataFrame ตามช่วงเปอร์เซ็นไทล์
    df_0_25 = df[df['f1'] <= percentile_25]
    df_25_75 = df[(df['f1'] > percentile_25) & (df['f1'] <= percentile_75)]
    df_75_100 = df[df['f1'] > percentile_75]

    # สุ่มตัวอย่างจากแต่ละช่วง
    sample_0_25 = df_0_25.sample(n=25, random_state=2024)
    sample_25_75 = df_25_75.sample(n=10, random_state=2024)
    sample_75_100 = df_75_100.sample(n=5, random_state=2024)
    print("Percentile 0-25")
    for _, row in df_0_25.iloc[:].iterrows():
        print('\nF1:', row['f1'])
        print('Q:', row['question'])
        print("K:", df_dataset[df_dataset["question"]==row["question"]]["knowledges"].values[0])
        print('A:', row['answer'])
        print('R:', row['response'])
        print()

# GPT-4o

In [12]:
MODEL_NAME = 'typhoon2-qwen2.5-7b-instruct-finetune-dataset-v1-exam-thailegal-with-v1+exam+thailegal-validation-set'
FIND_BEST_MODEL = False

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/25 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/13 [00:00<?, ?it/s]

done in 23.41 seconds, 8.54 sentences/sec
Precision: 0.8751378059387207
Recall:    0.8733235001564026
F1:        0.8731451630592346


100%|██████████| 200/200 [00:00<00:00, 527.87it/s]


Rouge
rouge1: 0.6886397151951013
rouge2: 0.639519961457615
rougeL: 0.6616390048621832


In [67]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat'
FIND_BEST_MODEL = False

MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, MODEL_NAME)
if FIND_BEST_MODEL:
    MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}.csv')
df = pd.read_csv(INFERECE_RESULT_PATH)
df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)

print('Q:', df.iloc[0]['question'])
print('A:', df.iloc[0]['answer'])
print('R:', df.iloc[0]['response'])
print()
for _, row in df.sample(4, random_state=0).iterrows():
    print('Q:', row['question'])
    print('A:', row['answer'])
    print('R:', row['response'])
    print()

Q: ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545 คำว่า 'สินค้า' หมายถึงอะไร
A: 'สินค้า' หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย
R: ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545, คำว่า "สินค้า" หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย

Q: ในกรณีที่โรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นก่อให้เกิดความเสียหาย บุคคลที่ประสบความเสียหายสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้หรือไม่
A: ใช่, บุคคลที่ประสบความเสียหายจากโรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้ตามมาตรา 435
R: ใช่ครับ/ค่ะ ในกรณีที่โรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นก่อให้เกิดความเสียหาย บุคคลที่ประสบความเสียหายสามารถเรียกร้องให้ผู้ครอบครองหรือเจ้าของสิ่งปลูกสร้างจัดการบำบัดปัดป้องภยันตรายได้ โดยตามมาตรา 435 ของประมวลกฎหมายแพ่งและพาณิชย์

Q: คำฟ้องในการดำเนินคดีแบบกลุ่มต้องมีลักษณะอย่างไร?
A: คำฟ้องต้องทำเป็นหนังสือและแสดงโดยแจ้งชัดซึ่งสภาพแห่งข้อหา, คำขอบังคับ, รวมทั้งข้ออ้างที่อาศัยเป็นหลักแห่งข้อหาของโจทก์และของกลุ่มบุคคลที่มีลักษณะเดียวกับโจทก์ด้วย
R: คำฟ้องในการดำเนินคดีแบบก

# SeaLLMs-v3-7B-Chat-finetune-dataset-v1

In [ ]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-dataset-v1'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

In [5]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-dataset-v1'
FIND_BEST_MODEL = True

MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, MODEL_NAME)
if FIND_BEST_MODEL:
    MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}.csv')
df = pd.read_csv(INFERECE_RESULT_PATH)
df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)

print('Q:', df.iloc[0]['question'])
print('A:', df.iloc[0]['answer'])
print('R:', df.iloc[0]['response'])
print()
for _, row in df.sample(4, random_state=0).iterrows():
    print('Q:', row['question'])
    print('A:', row['answer'])
    print('R:', row['response'])
    print()

Q: ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545 คำว่า 'สินค้า' หมายถึงอะไร
A: 'สินค้า' หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย
R: หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย

Q: ในกรณีที่โรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นก่อให้เกิดความเสียหาย บุคคลที่ประสบความเสียหายสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้หรือไม่
A: ใช่, บุคคลที่ประสบความเสียหายจากโรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้ตามมาตรา 435
R: สามารถเรียกร้องได้ โดยอาศัยอำนาจตามมาตรา 435

Q: คำฟ้องในการดำเนินคดีแบบกลุ่มต้องมีลักษณะอย่างไร?
A: คำฟ้องต้องทำเป็นหนังสือและแสดงโดยแจ้งชัดซึ่งสภาพแห่งข้อหา, คำขอบังคับ, รวมทั้งข้ออ้างที่อาศัยเป็นหลักแห่งข้อหาของโจทก์และของกลุ่มบุคคลที่มีลักษณะเดียวกับโจทก์ด้วย
R: คำฟ้องต้องทำเป็นหนังสือและแสดงโดยแจ้งชัดซึ่งสภาพแห่งข้อหา, คำขอบังคับ, รวมทั้งข้ออ้างที่อาศัยเป็นหลักแห่งข้อหาของโจทก์และของกลุ่มบุคคลที่มีลักษณะเดียวกับโจทก์ด้วย

Q: กรณีที่มีคำพิพากษาให้ลูกหนี้ต้องรื้อถอนสิ่งปลูกสร้าง แต่ลูกหนี้ไม่ดำเนินการ จะมีการดำเนินการอย่างไร
A: เจ

In [9]:
question = 'การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?'
df.query('question == @question')['response']

202    การประดิษฐ์ที่ไม่เป็นงานที่ปรากฏอยู่แล้ว
Name: response, dtype: object

# Llama-3-Typhoon-1.5-8B

In [81]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B'
FIND_BEST_MODEL = False

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/52 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 110.35 seconds, 3.74 sentences/sec

Bert Score
Precision: 0.7588768601417542
Recall:    0.8262055516242981
F1:        0.790398895740509


100%|██████████| 413/413 [00:01<00:00, 237.37it/s]


Rouge
rouge1: 0.4840635047940737
rouge2: 0.43111772172287466
rougeL: 0.4410674408014457

F1: 0.478336364030838
Q: หากคนต่างด้าวได้รับอนุญาตให้มีที่ดินเพื่อการศาสนา จะสามารถมีที่ดินได้ไม่เกินกี่ไร่?
A: สามารถมีที่ดินได้ไม่เกิน 1 ไร่
R: ตามประมวลกฎหมายที่ดิน พ.ศ. 2497 (ฉบับปรับปรุง), หมวด 8 (การกำหนดสิทธิในที่ดินของคนต่างด้าว), มาตรา 87 กำหนดว่า "ที่ดินที่จะพึงอนุญาตให้" สำหรับการใช้เพื่อการศาสนายังไม่มีการกำหนดจำนวน. ดังนั้น, ไม่สามารถระบุได้ว่าคนต่างด้าวสามารถมีที่ดินเพื่อการศาสนาได้ไม่เกินกี่ไร่. คำถามนี้ไม่ได้รวมถึงการใช้เพื่อการอุตสาหกรรม, การเกษตรกรรม, หรือการใช้เพื่อการพาณิชยกรรมที่มีการจำกัดจำนวนไว้แล้ว.

Reference(s):
ประมวลกฎหมายที่ดิน พ.ศ. 2497 (ฉบับปรับปรุง) - หมวด 8 (การกำหนดสิทธิในที่ดินของคนต่างด้าว), มาตรา 87 (7) ที่ใช้เพื่อการศาสนา ไม่เกิน 1 ไร่
ประมวลกฎหมายที่ดิน พ.ศ. 2497 (ฉบับปรับปรุง) - หมวด 8 (การกำหนดสิทธิในที่ดินของคนต่างด้าว), มาตรา 87 (4) ที่ใช้เพื่อเกษตรกรรม ครอบครัวละไม่เกิน 10 ไร่
ประมวลกฎหมายที่ดิน พ.ศ. 2497 (ฉบับปรับปรุง) - หมวด 8 (การกำหนดสิทธิในที่ดินของ

In [70]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B'
FIND_BEST_MODEL = False

MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, MODEL_NAME)
if FIND_BEST_MODEL:
    MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}.csv')
df = pd.read_csv(INFERECE_RESULT_PATH)
df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)

print('Q:', df.iloc[0]['question'])
print('A:', df.iloc[0]['answer'])
print('R:', df.iloc[0]['response'])
print()
for _, row in df.sample(4, random_state=0).iterrows():
    print('Q:', row['question'])
    print('A:', row['answer'])
    print('R:', row['response'])
    print()

Q: ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545 คำว่า 'สินค้า' หมายถึงอะไร
A: 'สินค้า' หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย
R: ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545, "สินค้า" หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย.</s

Q: ในกรณีที่โรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นก่อให้เกิดความเสียหาย บุคคลที่ประสบความเสียหายสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้หรือไม่
A: ใช่, บุคคลที่ประสบความเสียหายจากโรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้ตามมาตรา 435
R: ตามประมวลกฎหมายแพ่งและพาณิชย์, บุคคลใดจะประสพความเสียหายอันพึงเกิดจากโรงเรือนหรือสิ่งปลูกสร้างอย่างอื่นของผู้อื่น, บุคคลผู้นั้นชอบที่จะเรียกให้จัดการตามที่จำเป็นเพื่อบำบัดปัดป้องภยันตรายนั้นเสียได้ (มาตรา 435). ดังนั้น, ในกรณีที่โรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นก่อให้เกิดความเสียหาย, บุคคลที่ประสบความเสียหายนั้นสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้.

Q: คำฟ้องในการดำเนินคดีแบบกลุ่มต้องมีลักษณะอย่างไร?
A: คำฟ้องต้องทำเป็นหนังสือและแสดงโดยแจ้งชัดซึ่งสภาพแห่งข้อหา, 

# Llama-3-Typhoon-1.5-8B-finetune-dataset-v1

In [24]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-dataset-v1'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/49 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 42.35 seconds, 9.75 sentences/sec
Precision: 0.9019976258277893
Recall:    0.8978792428970337
F1:        0.8993901014328003


100%|██████████| 413/413 [00:00<00:00, 650.70it/s]



Rouge
rouge1: 0.7564285247469259
rouge2: 0.706490308342725
rougeL: 0.7294283655888049

F1: 0.6897110342979431
Q: เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?
K: ประมวลกฎหมายวิธีพิจารณาความอาญา - ภาค 1 (ข้อความเบื้องต้น) - ลักษณะ 4 (หมายเรียกและหมายอาญา) - หมวด 2 (หมายอาญา) - ส่วน 4 (หมายขัง หมายจำคุก หมายปล่อย)

มาตรา 75  เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษให้ปล่อยหรือมีคำวินิจฉัยให้ปล่อยตัวไปโดยมีเงื่อนไข หรือมีกฎหมายยกเว้นโทษหรือโทษจำคุกนั้นหมดไปโดยเหตุอื่น ให้ศาลออกหมายปล่อยผู้นั้นไป
A: ศาลจะออกหมายปล่อยผู้นั้นไป ไม่ว่าจะเป็นกรณีที่ผู้ต้องคำพิพากษาถูกจำครบกำหนดแล้ว, ได้รับพระราชทานอภัยโทษ, มีคำวินิจฉัยให้ปล่อยตัวไปโดยมีเงื่อนไข, หรือมีกฎหมายยกเว้นโทษหรือโทษจำคุกนั้นหมดไปโดยเหตุอื่น.
R: ศาลจะออกหมายปล่อยผู้นั้นไป


F1: 0.7015366554260254
Q: หากผู้ขับขี่ฝ่าฝืนคำสั่งของหัวหน้าเจ้าพนักงานจราจรตามมาตรา 43 ทวิ จะต้องระวางโทษอย่างไร
K: พระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565 - ลักษณะ 19 (บทกำหนด

In [22]:
INFERENCE_RESULT_DIR = '/project/lt200301-edubot/Capstone-TamTanai/inference_result'
df= pd.read_csv('/project/lt200301-edubot/Capstone-TamTanai/inference_result/inference_Llama-3-Typhoon-1.5-8B-finetune-dataset-v1_temperaure=0.1.csv')
df.iloc[365,4]

'ผู้ขับขี่รถจักรยานยนต์ที่ขับขี่โดยที่คนโดยสารไม่สวมหมวกนิรภัย จะถูกดำเนินการตามมาตรา 122 ของพระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565</s>\nReference(s):\nมาตรา 122 - ผู้ขับขี่รถจักรยานยนต์และคนโดยสารรถจักรยานยนต์ต้องสวมหมวกนิรภัย</s>\nมาตรา 122/1 - ห้ามมิให้ผู้ขับขี่รถจักรยานยนต์ตามมาตรา 122 ขับขี่รถจักรยานยนต์ในขณะที่คนโดยสารรถจักรยานยนต์มิได้สวมหมวกนิรภัย</s>\nมาตรา 122/2 - ลักษณะและวิธีการใช้หมวกนิรภัยตามมาตรา 122 และมาตรา 122/1 ต้องเป็นไปตามที่กำหนดในกฎกระทรวง</s>\nมาตรา 122/3 - ความในมาตรา 122 มิให้ใช้บังคับกับบุคคลตามที่กำหนดในกฎกระทรวง</s>\nมาตรา 122/4 - รัฐมนตรีหรือผู้ซึ่งได้รับมอบหมายจากรัฐมนตรีมีอำนาจออกกฎกระทรวงกำหนดรายละเอียดในมาตรา 122, มาตรา 122/1 และมาตรา 122/2</s>\nมาตรา 122/5 - ให้นำบทบัญญัติมาตรา 122 วรรคสองและวรรคสาม มาใช้บังคับกับมาตรา 122/1 และมาตรา 122/2 โดยอนุโลม</s>\nมาตรา 122/6 - ในกรณีที่ผู้ขับขี่รถจักรยานยนต์ตามมาตรา 122 หรือมาตรา 122/1 ได้กระทำการฝ่าฝืนหรือไม่ปฏิบัติตามมาตรา 122, มาตรา 122/1 หรือมาตรา 122/2 ถ้าผู้ขับขี่รถจักรยานยนต์ดังกล่าวไม่มีความผิด

In [10]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-dataset-v1'
FIND_BEST_MODEL = True

MODEL_NAME_OR_PATH = os.path.join(MODEL_DIR, MODEL_NAME)
if FIND_BEST_MODEL:
    MODEL_NAME_OR_PATH = find_best_model(MODEL_NAME)
INFERECE_RESULT_PATH = os.path.join(INFERENCE_RESULT_DIR, f'inference_{MODEL_NAME}.csv')
df = pd.read_csv(INFERECE_RESULT_PATH)
df['response'] = df['response_finetune'].apply(lambda x: x[: x.index('</s>')] if '</s>' in x else x)

print('Q:', df.iloc[0]['question'])
print('A:', df.iloc[0]['answer'])
print('R:', df.iloc[0]['response'])
print()
for _, row in df.sample(4, random_state=0).iterrows():
    print('Q:', row['question'])
    print('A:', row['answer'])
    print('R:', row['response'])
    print()

Q: ตามพระราชบัญญัติขายตรงและตลาดแบบตรง พ.ศ. 2545 คำว่า 'สินค้า' หมายถึงอะไร
A: 'สินค้า' หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย
R: หมายความว่า สิ่งของที่ผลิตหรือมีไว้เพื่อขาย

Q: ในกรณีที่โรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นก่อให้เกิดความเสียหาย บุคคลที่ประสบความเสียหายสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้หรือไม่
A: ใช่, บุคคลที่ประสบความเสียหายจากโรงเรือนหรือสิ่งปลูกสร้างของผู้อื่นสามารถเรียกร้องให้จัดการบำบัดปัดป้องภยันตรายได้ตามมาตรา 435
R: ใช่, สามารถเรียกร้องได้ ตามมาตรา 435

Q: คำฟ้องในการดำเนินคดีแบบกลุ่มต้องมีลักษณะอย่างไร?
A: คำฟ้องต้องทำเป็นหนังสือและแสดงโดยแจ้งชัดซึ่งสภาพแห่งข้อหา, คำขอบังคับ, รวมทั้งข้ออ้างที่อาศัยเป็นหลักแห่งข้อหาของโจทก์และของกลุ่มบุคคลที่มีลักษณะเดียวกับโจทก์ด้วย
R: คำฟ้องต้องทำเป็นหนังสือและแสดงโดยแจ้งชัดซึ่งสภาพแห่งข้อหา คำขอบังคับ รวมทั้งข้ออ้างที่อาศัยเป็นหลักแห่งข้อหาของโจทก์และของกลุ่มบุคคลที่มีลักษณะเดียวกับโจทก์ด้วย

Q: กรณีที่มีคำพิพากษาให้ลูกหนี้ต้องรื้อถอนสิ่งปลูกสร้าง แต่ลูกหนี้ไม่ดำเนินการ จะมีการดำเนินการอย่างไร
A: เจ้าพนักงานบ

In [14]:
question = 'เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?'
df.query('question == @question')['response'].iloc[0]

'ศาลจะออกหมายปล่อยผู้นั้นไป'

# SeaLLMs-v3-7B-Chat-finetune-v1-rslora

In [30]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-rslora'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 45.28 seconds, 9.12 sentences/sec

Bert Score
Precision: 0.8783004879951477
Recall:    0.8789315223693848
F1:        0.8778615593910217


100%|██████████| 413/413 [00:00<00:00, 580.74it/s]


Rouge
rouge1: 0.7207866515289331
rouge2: 0.6571877005040551
rougeL: 0.6791905607094811


# Llama-3-Typhoon-1.5-8B-finetune-v1-rslora

In [31]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-rslora'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 53.07 seconds, 7.78 sentences/sec

Bert Score
Precision: 0.8632537722587585
Recall:    0.8708348274230957
F1:        0.8662403225898743


100%|██████████| 413/413 [00:00<00:00, 522.36it/s]


Rouge
rouge1: 0.6873096897389038
rouge2: 0.6182542844913157
rougeL: 0.6377499606716913


In [26]:
!nvidia-smi

Thu Aug 22 00:58:41 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:03:00.0 Off |                    0 |
| N/A   37C    P0    59W / 400W |  35371MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   

# SeaLLMs-v3-7B-Chat-finetune-v1-orpo

In [5]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-orpo'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 46.95 seconds, 8.80 sentences/sec

Bert Score
Precision: 0.8885554671287537
Recall:    0.867139458656311
F1:        0.8768428564071655


100%|██████████| 413/413 [00:00<00:00, 636.67it/s]



Rouge
rouge1: 0.7226628635795481
rouge2: 0.6692079146354061
rougeL: 0.6874366230373153

F1: 0.6284034848213196
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: ไม่เป็นงานที่ปรากฏอยู่แล้ว, ไม่มีการเปิดเผยสาระสำคัญหรือรายละเอียดในเอกสารหรือสิ่งพิมพ์ที่เผยแพร่อยู่แล้ว, ไม่เคยได้รับสิทธิบัตรหรืออนุสิทธิบัตร, ไม่เคยมีการเปิดเผยสาระสำคัญหรือรายละเอียดโดยผู้ประดิษฐ์, และไม่เคยมีการเปิดเผยสาระสำคัญหรือรายละเอียดภายในสิบสองเดือนก่อนการขอรับสิทธิบัตร


F1: 0.6406024098396301
Q: บุคคลใดที่รับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ เพื่อยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ จะมีความผิดตามมาตราใด
A: มีความผิดตามมาตรา 172 ซึ่งระบุว่าเจ้าหนี้หรือผู้แทนเจ้าหนี้คนใด เรียกหรือรับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ ไว้เป็นประโยชน์แก่ตนเองหรือผู้อื่น เพื่อที่จะยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ หรือการขอปลดจากล้มละลาย มีความผิดต้องระวางโทษปรับไม่เกินห้าเท่าของราคาผลประโยชน์อันมิควรได้นั้น
R: บุ

# Llama-3-Typhoon-1.5-8B-finetune-v1-orpo

In [6]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-orpo'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 46.11 seconds, 8.96 sentences/sec

Bert Score
Precision: 0.888351559638977
Recall:    0.8838481903076172
F1:        0.8853016495704651


100%|██████████| 413/413 [00:00<00:00, 589.85it/s]



Rouge
rouge1: 0.7165558561654273
rouge2: 0.6613574032989429
rougeL: 0.6796088436528622

F1: 0.6480733156204224
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว ไม่ว่าจะเป็นการประดิษฐ์ที่มีอยู่ในราชอาณาจักร, เปิดเผยอยู่ในเอกสารหรือสิ่งพิมพ์, ได้รับสิทธิบัตรหรืออนุสิทธิบัตรแล้ว, มีผู้ขอรับสิทธิบัตรหรืออนุสิทธิบัตรไว้แล้วนอกราชอาณาจักรเป็นเวลาสิบแปดเดือนก่อนขอรับสิทธิบัตร, หรือถูกประกาศโฆษณาแล้วในราชอาณาจักร


F1: 0.6865684986114502
Q: หากผู้ยื่นคำขอสิทธิบัตรไม่ปฏิบัติตามคำสั่งของพนักงานเจ้าหน้าที่ในการส่งเอกสารเพิ่มเติม จะถือว่าอย่างไรตามพระราชบัญญัติสิทธิบัตร (ฉบับที่ 3) พ.ศ. 2542?
A: ตามมาตรา 27 ของพระราชบัญญัติสิทธิบัตร (ฉบับที่ 3) พ.ศ. 2542 หากผู้ยื่นคำขอสิทธิบัตรไม่ปฏิบัติตามคำสั่งของพนักงานเจ้าหน้าที่ในการส่งเอกสารเพิ่มเติมภายในกำหนดเวลา จะถือว่าผู้นั้นได้ละทิ้งคำขอสิทธิบัตร
R: ถือว่าละทิ้งคำขอรับสิทธิบัตร


F1: 0.6897110342979431
Q: เมื่อผู้ต้องคำพิพากษาให้จำ

## SeaLLM3-7B-Chat-finetune-v1-cpo

In [5]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-cpo'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 55.35 seconds, 7.46 sentences/sec

Bert Score
Precision: 0.8845152258872986
Recall:    0.8687872886657715
F1:        0.8756171464920044


100%|██████████| 413/413 [00:00<00:00, 600.54it/s]



Rouge
rouge1: 0.7097426134490175
rouge2: 0.6526937685997056
rougeL: 0.6720966223140259

F1: 0.585040271282196
Q: ในกรณีที่มีการยกเลิกมาตราใดในพระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้าว (ฉบับที่ 2) พ.ศ. 2561
A: มาตราที่ถูกยกเลิกในพระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้าว (ฉบับที่ 2) พ.ศ. 2561 คือ มาตรา 24
R: มาตรา 24, มาตรา 25, มาตรา 27 และมาตรา 28, มาตรา 30, มาตรา 31, มาตรา 32 และมาตรา 33, มาตรา 36 และมาตรา 39, มาตรา 40, มาตรา 41, มาตรา 42, มาตรา 43, มาตรา 44 และมาตรา 45, มาตรา 48, มาตรา 49 และมาตรา 50, มาตรา 51 และมาตรา 52, มาตรา 53 และมาตรา 54, มาตรา 56 และมาตรา 57, มาตรา 59, มาตรา 60 และมาตรา 61, มาตรา 62 และมาตรา 63, มาตรา 64 และมาตรา 65, มาตรา 66 และมาตรา 67, มาตรา 68 และมาตรา 69, มาตรา 70, มาตรา 71 และมาตรา 72, มาตรา 73 และมาตรา 74, มาตรา 76 และมาตรา 77, มาตรา 79, มาตรา 80 และมาตรา 81, มาตรา 82, มาตรา 83 และมาตรา 84, มาตรา 86 และมาตรา 87, มาตรา 88 และมาตรา 89, มาตรา 90 และมาตรา 91, มาตรา 94 และมาตรา 95, มาตรา 97, มาตรา 98 และมาตรา 99, มาตรา 100, มาตรา 101 และมาตรา 102

## Llama-3-Typhoon-1.5-8B-finetune-v1-cpo

In [6]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-cpo'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 42.71 seconds, 9.67 sentences/sec

Bert Score
Precision: 0.8816362023353577
Recall:    0.8716322779655457
F1:        0.8758301138877869


100%|██████████| 413/413 [00:00<00:00, 742.18it/s]


Rouge
rouge1: 0.6933732607040212
rouge2: 0.6415193199422929
rougeL: 0.665844309114836

F1: 0.6215644478797913
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว, ไม่ได้รับสิทธิบัตรหรืออนุสิทธิบัตรแล้ว, ไม่มีผู้ขอรับสิทธิบัตรหรืออนุสิทธิบัตร, ไม่ได้ประกาศโฆษณา, หรือการเปิดเผยสาระสำคัญหรือรายละเอียดที่เกิดขึ้นหรือเป็นผลมาจากการกระทำอันมิชอบด้วยกฎหมาย หรือการเปิดเผยสาระสำคัญหรือรายละเอียดโดยผู้ประดิษฐ์รวมทั้งการแสดงผลงานของผู้ประดิษฐ์และการเปิดเผยสาระสำคัญหรือรายละเอียดดังกล่าวได้กระทำภายในสิบสองเดือนก่อนที่จะมีการขอรับสิทธิบัตร


F1: 0.6537578105926514
Q: หากผู้ตั้งมูลนิธิเสียชีวิตก่อนทำข้อกำหนดบางอย่าง จะดำเนินการอย่างไร
A: หากผู้ตั้งมูลนิธิเสียชีวิตก่อนทำข้อกำหนดบางอย่าง ผู้มีส่วนได้เสียหรือพนักงานอัยการสามารถร้องขอต่อศาลเพื่อกำหนดข้อกำหนดเหล่านั้นได้
R: ศาลจะต้องกำหนดให้


F1: 0.6815969347953796
Q: บุคคลใดที่รับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด 

## Llama-3-typhoon fine tune dataset v1 temp=0.1

In [11]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-dataset-v1'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/49 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 42.23 seconds, 9.78 sentences/sec
Precision: 0.9019976258277893
Recall:    0.8978792428970337
F1:        0.8993901014328003


100%|██████████| 413/413 [00:00<00:00, 729.65it/s]


Rouge
rouge1: 0.7564285247469259
rouge2: 0.706490308342725
rougeL: 0.7294283655888049

F1: 0.6897110939025879
Q: เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?
A: ศาลจะออกหมายปล่อยผู้นั้นไป ไม่ว่าจะเป็นกรณีที่ผู้ต้องคำพิพากษาถูกจำครบกำหนดแล้ว, ได้รับพระราชทานอภัยโทษ, มีคำวินิจฉัยให้ปล่อยตัวไปโดยมีเงื่อนไข, หรือมีกฎหมายยกเว้นโทษหรือโทษจำคุกนั้นหมดไปโดยเหตุอื่น.
R: ศาลจะออกหมายปล่อยผู้นั้นไป


F1: 0.7015365362167358
Q: หากผู้ขับขี่ฝ่าฝืนคำสั่งของหัวหน้าเจ้าพนักงานจราจรตามมาตรา 43 ทวิ จะต้องระวางโทษอย่างไร
A: ต้องระวางโทษสูงกว่าที่กำหนดไว้ในกฎหมายว่าด้วยยาเสพติดให้โทษหรือกฎหมายว่าด้วยวัตถุที่ออกฤทธิ์ต่อจิตและประสาทอีกหนึ่งในสาม และให้ศาลสั่งพักใช้ใบอนุญาตขับขี่ของผู้นั้นมีกำหนดไม่น้อยกว่าหกเดือน หรือเพิกถอนใบอนุญาตขับขี่
R: ต้องชำระค่าปรับเป็นพินัยไม่เกินสี่พันบาท


F1: 0.7108224034309387
Q: ในกรณีที่ทรัพย์สินที่เป็นวัตถุของสัญญาต่างตอบแทนสูญหายหรือเสียหายโดยไม่มีความผิดของลูกหนี้ ผลที่ตามมาจะเป็นอย่างไร?
A: การสูญหายหรือเสียหายนั้นตกเป็นภ

## Llama-3-typhoon fine tune dataset v1 orpo lr=1e-5

In [10]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-orpo-lr=1e-5'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 44.44 seconds, 9.29 sentences/sec

Bert Score
Precision: 0.8841402530670166
Recall:    0.8902860283851624
F1:        0.886557936668396


100%|██████████| 413/413 [00:00<00:00, 615.90it/s]


Rouge
rouge1: 0.7259542731151523
rouge2: 0.6711084168579431
rougeL: 0.6910870428124604

F1: 0.5827566385269165
Q: บุคคลใดที่จะต้องเสียเงินเพิ่มภาษีอากรตามประมวลรัษฎากรและชำระเงินเพิ่มภาษีอากรแล้วจะไม่ต้องรับผิดเสียเงินเพิ่มอีก
A: บุคคลที่ต้องเสียเงินเพิ่มภาษีอากรตามบทบัญญัติแห่งประมวลรัษฎากรและยินยอมและชำระเงินเพิ่มภาษีอากรตามหลักเกณฑ์ที่กำหนดในกฎกระทรวงแล้ว
R: บุคคลที่ได้ชำระเงินเพิ่มภาษีอากรไปแล้วตามมาตรา 12 ทวิ, มาตรา 13, มาตรา 14, มาตรา 15, มาตรา 16, มาตรา 17, มาตรา 18, มาตรา 19, มาตรา 20, มาตรา 21, มาตรา 22, มาตรา 23, มาตรา 24, มาตรา 25, มาตรา 26, มาตรา 27, มาตรา 28, มาตรา 29, มาตรา 30, มาตรา 31, มาตรา 32, มาตรา 33, มาตรา 34, มาตรา 35, มาตรา 36, มาตรา 37, มาตรา 38, มาตรา 39, มาตรา 40, มาตรา 41, มาตรา 42, มาตรา 43, มาตรา 44, มาตรา 45, มาตรา 46, มาตรา 47, มาตรา 48, มาตรา 49, มาตรา 50, มาตรา 51, มาตรา 52, มาตรา 53, มาตรา 54, มาตรา 55, มาตรา 56, มาตรา 57, มาตรา 58, มาตรา 59, มาตรา 60, มาตรา 61, มาตรา 62, มาตรา 63, มาตรา 64, มาตรา 65, มาตรา 66, มาตรา 67, มาตรา 68, มาตรา 69, มาตรา 70, 

## SEALLM3 7B fine tune dataset v1 orpo lr=1e-5

In [11]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-orpo-lr=1e-5'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 40.39 seconds, 10.23 sentences/sec

Bert Score
Precision: 0.8903638124465942
Recall:    0.8844795227050781
F1:        0.8866779804229736


100%|██████████| 413/413 [00:00<00:00, 598.45it/s]


Rouge
rouge1: 0.7470734235518017
rouge2: 0.6924995065036236
rougeL: 0.710416417718795

F1: 0.6251110434532166
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ที่ไม่เป็นงานที่ปรากฏอยู่แล้ว, ไม่เป็นการประดิษฐ์ที่ได้รับการเปิดเผยสาระสำคัญหรือรายละเอียด, ไม่เป็นการประดิษฐ์ที่ได้รับสิทธิบัตรหรืออนุสิทธิบัตรแล้ว, ไม่เป็นการประดิษฐ์ที่มีการขอรับสิทธิบัตรหรอนุสิทธิบัตรไว้แล้วนอกราชอาณาจักร, และไม่เป็นการประดิษฐ์ที่มีการเปิดเผยสาระสำคัญหรือรายละเอียดโดยผิดกฎหมาย


F1: 0.6406024098396301
Q: บุคคลใดที่รับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ เพื่อยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ จะมีความผิดตามมาตราใด
A: มีความผิดตามมาตรา 172 ซึ่งระบุว่าเจ้าหนี้หรือผู้แทนเจ้าหนี้คนใด เรียกหรือรับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ ไว้เป็นประโยชน์แก่ตนเองหรือผู้อื่น เพื่อที่จะยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ หรือการขอปลดจากล้มละลาย มีความผิดต้องระวางโทษปรับไม่เกินห้าเท่าของราคาผลประโ

## Llama-3-typhoon fine tune dataset v1 max token = 1000

In [7]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-dataset-v1'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/49 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 45.84 seconds, 9.01 sentences/sec
Precision: 0.8919538259506226
Recall:    0.8895781636238098
F1:        0.8900858163833618


100%|██████████| 413/413 [00:00<00:00, 710.80it/s]


Rouge
rouge1: 0.7350600068686499
rouge2: 0.6858221727344003
rougeL: 0.7090695698142288

F1: 0.6814032793045044
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว ไม่ว่าจะเป็นการประดิษฐ์ที่มีหรือใช้แพร่หลายอยู่แล้ว, ที่ได้มีการเปิดเผยสาระสำคัญหรือรายละเอียด, ที่ได้รับสิทธิบัตรหรืออนุสิทธิบัตร, ที่มีผู้ขอรับสิทธิบัตรหรืออนุสิทธิบัตรไว้แล้ว, หรือที่มีการประกาศโฆษณาแล้ว


F1: 0.6825100779533386
Q: ในกรณีที่ลูกจ้างทำงานล่วงเวลาเกินกว่าวันละแปดชั่วโมง นายจ้างต้องจ่ายค่าตอบแทนเพิ่มเติมอย่างน้อยกี่เท่าของอัตราค่าจ้าง?
A: นายจ้างต้องจ่ายค่าตอบแทนไม่น้อยกว่าหนึ่งเท่าครึ่งของอัตราค่าจ้าง
R: นายจ้างต้องจ่ายค่าตอบแทนไม่น้อยกว่าหนึ่งเท่าครึ่งของอัตราค่าจ้างต่อชั่วโมงในวันทำงานตามจำนวนชั่วโมงที่ทำเกินสำหรับลูกจ้างรายวันและลูกจ้างรายชั่วโมงหรือไม่น้อยกว่าหนึ่งเท่าครึ่งของอัตราค่าจ้างต่อหน่วยในวันทำงานตามจำนวนผลงานที่ทำได้ในชั่วโมงที่ทำเกินสำหรับลูกจ้างซึ่งได

## SEALLM3 7B fine tune dataset v1 orpo batch size = 8

In [5]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-orpo-batch-size=8'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/49 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 55.57 seconds, 7.43 sentences/sec
Precision: 0.8925812244415283
Recall:    0.883310079574585
F1:        0.8870972990989685


100%|██████████| 413/413 [00:00<00:00, 572.20it/s]


Rouge
rouge1: 0.746919675173279
rouge2: 0.6960869665349596
rougeL: 0.711542578222346

F1: 0.6640632748603821
Q: ใครเป็นผู้รับผิดชอบในการแต่งตั้งพนักงานเจ้าหน้าที่ตามพระราชบัญญัติการขุดดินและถมดิน พ.ศ. 2543
A: เจ้าพนักงานท้องถิ่นเป็นผู้มีอำนาจในการแต่งตั้งพนักงานเจ้าหน้าที่ตามพระราชบัญญัตินี้
R: พนักงานเจ้าหน้าที่ได้รับการแต่งตั้งจากรัฐมนตรี, เจ้าพนักงานท้องถิ่น, คณะเทศมนตรี, คณะผู้ว่าราชการกรุงเทพมหานคร, คณะผู้ว่าราชการจังหวัด, คณะผู้บริหารองค์การบริหารส่วนตำบล, คณะผู้บริหารองค์การบริหารส่วนตำบลหรือองค์การบริหารส่วนตำบล, คณะผู้บริหารองค์การบริหารส่วนตำบลหรือองค์การบริหารส่วนตำบลและองค์การบริหารส่วนตำบล, คณะผู้บริหารองค์การบริหารส่วนตำบลหรือองค์การบริหารส่วนตำบลและองค์การบริหารส่วนตำบลและองค์การบริหารส่วนตำบลและองค์การบริหารส่วนตำบลและองค์การบริหารส่วนตำบลและองค์การบริหารส่วนตำบลและองค์การบร将成为จะต้องแตกต่างกัน


F1: 0.6745724678039551
Q: การประกาศโฆษณาคำขอรับสิทธิบัตรมีค่าใช้จ่ายเท่าไหร่
A: 500 บาท
R: การประกาศโฆษณาคำขอรับสิทธิบัตรมีค่าใช้จ่าย 500 บาท


F1: 0.6871125102043152
Q: ค่าธรร

## SEALLM3 7B fine tune dataset v1 orpo lr=1e-5 batch size = 8

In [6]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-orpo-batchsize=8-lr=1e-5'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/49 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 39.70 seconds, 10.40 sentences/sec
Precision: 0.891607940196991
Recall:    0.8776927590370178
F1:        0.8837427496910095


100%|██████████| 413/413 [00:00<00:00, 625.53it/s]


Rouge
rouge1: 0.7311317673078945
rouge2: 0.6796842030034087
rougeL: 0.6993300088514437

F1: 0.6476759314537048
Q: เมื่อศาลสั่งรับคำร้องขอฟื้นฟูกิจการแล้ว จะต้องดำเนินการอย่างไร?
A: เมื่อศาลสั่งรับคำร้องขอฟื้นฟูกิจการแล้ว จะต้องดำเนินการไต่สวนเป็นการด่วน ประกาศคำสั่งรับคำร้องขอและวันเวลานัดไต่สวนในหนังสือพิมพ์รายวันที่แพร่หลายหนึ่งฉบับล่วงหน้าก่อนวันนัดไต่สวนไม่น้อยกว่าสิบห้าวัน และส่งสำเนาคำร้องขอและแจ้งวันเวลานัดไต่สวนแก่ลูกหนี้ เจ้าหนี้ทั้งหลาย และเจ้าพนักงานพิทักษ์ทรัพย์ รวมถึงแจ้งให้นายทะเบียนหุ้นส่วนบริษัทหรือนายทะเบียนนิติบุคคลที่เกี่ยวข้อง
R: ต้องดำเนินการไต่สวนเป็นการด่วน


F1: 0.6871125102043152
Q: ค่าธรรมเนียมสำหรับการคัดสำเนาเอกสารต่อหน้าเท่าไหร่
A: 10 บาท
R: ค่าธรรมเนียมคัดสำเนาเอกสารคือ 10 บาทต่อหน้า


F1: 0.6897110342979431
Q: เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?
A: ศาลจะออกหมายปล่อยผู้นั้นไป ไม่ว่าจะเป็นกรณีที่ผู้ต้องคำพิพากษาถูกจำครบกำหนดแล้ว, ได้รับพระราชทานอภัยโทษ, มีคำวินิจฉัยให้ปล่อยตัวไปโดยมีเงื่อนไข, หรือ

## Llama-3-typhoon fine tune dataset v1 orpo batch size = 8

In [7]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-orpo-batchsize=8'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 40.65 seconds, 10.16 sentences/sec
Precision: 0.8879120945930481
Recall:    0.8785220980644226
F1:        0.8824071288108826


100%|██████████| 413/413 [00:00<00:00, 707.05it/s]


Rouge
rouge1: 0.7056870490491588
rouge2: 0.6523153254680129
rougeL: 0.6778467750168338

F1: 0.6897110342979431
Q: เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?
A: ศาลจะออกหมายปล่อยผู้นั้นไป ไม่ว่าจะเป็นกรณีที่ผู้ต้องคำพิพากษาถูกจำครบกำหนดแล้ว, ได้รับพระราชทานอภัยโทษ, มีคำวินิจฉัยให้ปล่อยตัวไปโดยมีเงื่อนไข, หรือมีกฎหมายยกเว้นโทษหรือโทษจำคุกนั้นหมดไปโดยเหตุอื่น.
R: ศาลจะออกหมายปล่อยผู้นั้นไป


F1: 0.6923559904098511
Q: หากเจ้าพนักงานบังคับคดีกระทำการฝ่าฝืนต่อบทบัญญัติแห่งประมวลกฎหมายนี้ จะมีผลอย่างไร?
A: ความรับผิดทางละเมิดต่อลูกหนี้ตามคำพิพากษาหรือต่อบุคคลภายนอกเพื่อความเสียหายที่เกิดจากหรือเกี่ยวเนื่องกับการบังคับคดีโดยมิชอบจะตกอยู่แก่เจ้าหนี้ตามคำพิพากษา เว้นแต่ในกรณีเจ้าพนักงานบังคับคดีได้กระทำการฝ่าฝืนต่อบทบัญญัติแห่งประมวลกฎหมายนี้ ความรับผิดจะตกแก่เจ้าพนักงานบังคับคดี
R: ความรับผิดทางละเมิดจะตกอยู่แก่เจ้าหนี้ตามคำพิพากษา


F1: 0.701668918132782
Q: ค่าธรรมเนียมสำหรับการคัดสำเนาเอกสารต่อหน้าเท่าไหร่
A: 10 บาท
R: ค่าธรรมเนียมคือ 10 บ

## Llama-3-typhoon fine tune dataset v1 orpo lr=1e-5 batch size = 8

In [8]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-orpo-batchsize=8-lr=1e-5'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 43.97 seconds, 9.39 sentences/sec
Precision: 0.8894000053405762
Recall:    0.8886517286300659
F1:        0.888305127620697


100%|██████████| 413/413 [00:00<00:00, 660.03it/s]


Rouge
rouge1: 0.7270521742031791
rouge2: 0.6735496597012998
rougeL: 0.6953385522183717

F1: 0.6491954326629639
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว ไม่ว่าจะเป็นการประดิษฐ์ที่มีหรือใช้แพร่หลายอยู่แล้วในราชอาณาจักร, เปิดเผยสาระสำคัญหรือรายละเอียดในเอกสารหรือสิ่งพิมพ์, ได้รับสิทธิบัตรหรืออนุสิทธิบัตรแล้ว, มีผู้ขอรับสิทธิบัตรหรืออนุสิทธิบัตรไว้แล้วนอกราชอาณาจักรเป็นเวลาเกินสิบแปดเดือน, หรือมีการประกาศโฆษณาแล้วก่อนวันขอรับสิทธิบัตรในราชอาณาจักร


F1: 0.6595593094825745
Q: ค่าธรรมเนียมสำหรับการคัดสำเนาเอกสารต่อหน้าเท่าไหร่
A: 10 บาท
R: ค่าธรรมเนาในการคัดสำเนาเอกสารต่อหน้าเท่ากับ 10 บาท


F1: 0.6734180450439453
Q: กรณีที่รัฐบาลต้องการควบคุมการเช่าที่ดินเพื่อประกอบเกษตรกรรมประเภทใดๆ นอกจากการเช่านา เพื่อขจัดปัญหาการเอารัดเอาเปรียบเกษตรกร จะดำเนินการอย่างไรตามพระราชบัญญัติการเช่าที่ดินเพื่อเกษตรกรรม พ.ศ. 2524
A: ตามมาตรา 63 รัฐบาลมีอำนาจที่จะตราพระราชกฤษฎีกาเพื่อควบคุม

## Llama-3-typhoon fine tune dataset v1 orpo lr=1e-5 batch size = 8 epoch = 10

In [6]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-orpo-batchsize=8-lr=1e-5-epoch=10'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 61.72 seconds, 6.69 sentences/sec
Precision: 0.8775551915168762
Recall:    0.8838390111923218
F1:        0.8800019025802612


100%|██████████| 413/413 [00:00<00:00, 647.13it/s]



Rouge
rouge1: 0.7039095998772489
rouge2: 0.6461950598893079
rougeL: 0.6700467855199406

F1: 0.6490016579627991
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว ไม่ว่าจะเป็นการประดิษฐ์ที่มีหรือใช้แพร่หลายอยู่แล้ว, ที่ได้รับการเปิดเผยสาระสำคัญหรือรายละเอียด, ที่ได้รับสิทธิบัตรหรืออนุสิทธิบัตรแล้ว, มีผู้ขอรับสิทธิบัตรหรืออนุสิทธิบัตรไว้แล้วนอกราชอาณาจักร, หรือมีผู้ขอรับสิทธิบัตรหรืออนุสิทธิบัตรไว้แล้วและได้ประกาศโฆษณาแล้ว


F1: 0.6778722405433655
Q: การประกาศโฆษณาคำขอรับสิทธิบัตรมีค่าใช้จ่ายเท่าไหร่
A: 500 บาท
R: 500 บาทต่อครั้งตามบัญชีอัตราค่าธรรมเนียม


F1: 0.6973193287849426
Q: เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?
A: ศาลจะออกหมายปล่อยผู้นั้นไป ไม่ว่าจะเป็นกรณีที่ผู้ต้องคำพิพากษาถูกจำครบกำหนดแล้ว, ได้รับพระราชทานอภัยโทษ, มีคำวินิจฉัยให้ปล่อยตัวไปโดยมีเงื่อนไข, หรือมีกฎหมายยกเว้นโทษหรือโทษจำคุกนั้นหมดไปโดยเหตุอื่น.


## SEALLM3 7B fine tune dataset v1 orpo lr=1e-5 batch size = 8 epoch = 10

In [7]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-orpo-batchsize=8-lr=1e-5-epoch=10'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/49 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 38.66 seconds, 10.68 sentences/sec
Precision: 0.8914128541946411
Recall:    0.8777585625648499
F1:        0.8837510943412781


100%|██████████| 413/413 [00:00<00:00, 648.34it/s]


Rouge
rouge1: 0.731697034891994
rouge2: 0.6766167480599229
rougeL: 0.6966939493427163

F1: 0.6336523294448853
Q: ในกรณีที่มีการยกเลิกมาตราใดในพระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้าว (ฉบับที่ 2) พ.ศ. 2561
A: มาตราที่ถูกยกเลิกในพระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้าว (ฉบับที่ 2) พ.ศ. 2561 คือ มาตรา 24
R: ไม่รู้


F1: 0.6406024098396301
Q: บุคคลใดที่รับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ เพื่อยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ จะมีความผิดตามมาตราใด
A: มีความผิดตามมาตรา 172 ซึ่งระบุว่าเจ้าหนี้หรือผู้แทนเจ้าหนี้คนใด เรียกหรือรับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ ไว้เป็นประโยชน์แก่ตนเองหรือผู้อื่น เพื่อที่จะยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ หรือการขอปลดจากล้มละลาย มีความผิดต้องระวางโทษปรับไม่เกินห้าเท่าของราคาผลประโยชน์อันมิควรได้นั้น
R: บุคคลดังกล่าวจะมีความผิดตามมาตรา 172


F1: 0.6897110342979431
Q: เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?
A: ศาลจะออกหมายปล่อยผู้นั้นไป ไม

## SEALLM3 7B fine tune dataset v1 orpo lr=8e-6 batch size = 8 epoch = 10

In [6]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-orpo-batchsize=8-lr=8e-6-epoch=10'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/49 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 54.66 seconds, 7.56 sentences/sec
Precision: 0.8901476263999939
Recall:    0.8796117901802063
F1:        0.8841161131858826


100%|██████████| 413/413 [00:00<00:00, 581.81it/s]


Rouge
rouge1: 0.7319778834745901
rouge2: 0.6759399321744484
rougeL: 0.6982353359515435

F1: 0.5984339118003845
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ที่ไม่เป็นงานที่ปรากฏอยู่แล้ว โดยงานที่ปรากฏอยู่แล้วหมายถึงการประดิษฐ์ที่มีหรือใช้แพร่หลายอยู่แล้วในราชอาณาจักรก่อนวันขอรับสิทธิบัตร, มีการเปิดเผยสาระสำคัญหรือรายละเอียดในเอกสารหรือสิ่งพิมพ์ที่ได้เผยแพร่อยู่แล้ว, ได้รับสิทธิบัตรหรืออนุสิทธิบัตรแล้ว, มีการขอรับสิทธิบัตรหรืออนุสิทธิบัตรไว้แล้วนอกราชอาณาจักร, หรือมีการเปิดเผยสาระสำคัญหรือรายละเอียดโดยผิดกฎหมาย


F1: 0.6406024098396301
Q: บุคคลใดที่รับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ เพื่อยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ จะมีความผิดตามมาตราใด
A: มีความผิดตามมาตรา 172 ซึ่งระบุว่าเจ้าหนี้หรือผู้แทนเจ้าหนี้คนใด เรียกหรือรับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ ไว้เป็นประโยชน์แก่ตนเองหรือผู้อื่น เพื่อที่จะยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ หรือการขอปลดจาก

## Llama-3-typhoon fine tune dataset v1 orpo lr=8e-6 batch size = 8 epoch = 10

In [7]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-orpo-batchsize=8-lr=8e-6-epoch=10'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 44.77 seconds, 9.22 sentences/sec
Precision: 0.8785271644592285
Recall:    0.8887032270431519
F1:        0.882919430732727


100%|██████████| 413/413 [00:00<00:00, 668.14it/s]



Rouge
rouge1: 0.7161952007459391
rouge2: 0.6616342305123426
rougeL: 0.6828975332570684

F1: 0.6239704489707947
Q: การประกาศโฆษณาคำขอรับสิทธิบัตรมีค่าใช้จ่ายเท่าไหร่
A: 500 บาท
R: มีค่าใช้จ่าย 500 บาท
กำหนดตามบัญชีอัตราค่าธรรมเนียมในประมวลกฎหมายว่าด้วยสิทธิบัตร


F1: 0.6815969347953796
Q: บุคคลใดที่รับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ เพื่อยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ จะมีความผิดตามมาตราใด
A: มีความผิดตามมาตรา 172 ซึ่งระบุว่าเจ้าหนี้หรือผู้แทนเจ้าหนี้คนใด เรียกหรือรับหรือยินยอมที่จะรับทรัพย์สินหลักประกันหรือผลประโยชน์ใด ๆ ไว้เป็นประโยชน์แก่ตนเองหรือผู้อื่น เพื่อที่จะยินยอมหรือไม่คัดค้านในการขอประนอมหนี้ หรือการขอปลดจากล้มละลาย มีความผิดต้องระวางโทษปรับไม่เกินห้าเท่าของราคาผลประโยชน์อันมิควรได้นั้น
R: บุคคลดังกล่าวจะมีความผิดตามมาตรา 172 ของพระราชบัญญัติล้มละลาย (ฉบับที่ 10) พ.ศ. 2561


F1: 0.6817045211791992
Q: ในกรณีที่ลูกจ้างทำงานล่วงเวลาเกินกว่าวันละแปดชั่วโมง นายจ้างต้องจ่ายค่าตอบแทนเพิ่มเติมอย่างน้อยกี่เท่าของอัตราค่าจ้าง?
A: นายจ้างต้องจ่ายค่าตอบแทน

## Llama-3-Typhoon-1.5-8B-finetune-v1-orpo-batchsize=8-lr=1e-5-beta=0.5

In [5]:
MODEL_NAME = 'Llama-3-Typhoon-1.5-8B-finetune-v1-orpo-batchsize=8-lr=1e-5-beta=0.5'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/26 [00:00<?, ?it/s]

done in 57.28 seconds, 7.21 sentences/sec
Precision: 0.8781954646110535
Recall:    0.8811203837394714
F1:        0.8789274096488953


100%|██████████| 413/413 [00:00<00:00, 626.62it/s]



Rouge
rouge1: 0.6986819073665853
rouge2: 0.6417135978052508
rougeL: 0.6662598679392482

F1: 0.6559622287750244
Q: การประดิษฐ์ที่ถือว่าเป็นการประดิษฐ์ขึ้นใหม่ต้องไม่เป็นอย่างไร?
A: การประดิษฐ์ขึ้นใหม่ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว
R: การประดิษฐ์ต้องไม่เป็นงานที่ปรากฏอยู่แล้ว ไม่ว่าจะเป็นการประดิษฐ์ที่มีหรือใช้แพร่หลายอยู่แล้วในราชอาณาจักร, เปิดเผยสาระสำคัญหรือรายละเอียดในเอกสารหรือสิ่งพิมพ์, ได้รับสิทธิบัตรหรืออนุสิทธิบัตรแล้ว, มีผู้ขอรับสิทธิบัตรหรืออนุสิทธิบัตรไว้แล้วนอกราชอาณาจักรเป็นเวลาเกินสิบแปดเดือนก่อนวันขอรับสิทธิบัตร, หรือมีการประกาศโฆษณาแล้ว


F1: 0.6595593094825745
Q: ค่าธรรมเนียมสำหรับการคัดสำเนาเอกสารต่อหน้าเท่าไหร่
A: 10 บาท
R: ค่าธรรมเนาในการคัดสำเนาเอกสารต่อหน้าเท่ากับ 10 บาท


F1: 0.6973193287849426
Q: เมื่อผู้ต้องคำพิพากษาให้จำคุกถูกจำครบกำหนดแล้วหรือได้รับพระราชทานอภัยโทษ ศาลจะดำเนินการอย่างไร?
A: ศาลจะออกหมายปล่อยผู้นั้นไป ไม่ว่าจะเป็นกรณีที่ผู้ต้องคำพิพากษาถูกจำครบกำหนดแล้ว, ได้รับพระราชทานอภัยโทษ, มีคำวินิจฉัยให้ปล่อยตัวไปโดยมีเงื่อนไข, หรือมีกฎหมายยกเว้นโทษหรือโ

## SeaLLMs-v3-7B-Chat-finetune-v1-orpo-batchsize=8-lr=1e-5-beta=0.5

In [6]:
MODEL_NAME = 'SeaLLMs-v3-7B-Chat-finetune-v1-orpo-batchsize=8-lr=1e-5-beta=0.5'
FIND_BEST_MODEL = True

run_evaluation(MODEL_NAME, FIND_BEST_MODEL)

calculating scores...
computing bert embedding.


  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.23 GiB. GPU 